## Get topics from each subreddit

In [1]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.cluster import KMeans

/home/biso/Uni/corsi/uni_venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = pd.read_csv('data/final_dataset.csv')

In [3]:
dataset

,text,labels,id,author,subreddit,created_utc
0,My favourite food is anything I didn't have to...,[27],eebbqej,Rockzilla78,KitchenConfidential,2019-01-18
1,"Now if he does off himself, everyone will thin...",[27],ed00q6i,cocaineluna,morbidquestions,2019-01-01
2,WHY THE FUCK IS BAYLESS ISOING,[2],eezlygj,minnesotagoat_,timberwolves,2019-01-26
3,To make her feel threatened,[14],ed7ypvh,AlexNic1013,askwomenadvice,2019-01-04
4,Dirty Southern Wankers,[3],ed0bdzj,having_a_nap,london,2019-01-01
...,...,...,...,...,...,...
54258,Thanks. I was diagnosed with BP 1 after the ho...,[15],efeeasc,scaredyk4t,bipolar,2019-01-30
54259,Well that makes sense.,[4],ef9c7s3,LX_Emergency,ABoringDystopia,2019-01-29
54260,Daddy issues [NAME],[27],efbiugo,your_ex_girlfriend-,SquaredCircle,2019-01-30
54261,So glad I discovered that subreddit a couple m...,[0],efbvgp9,Paladin-Arda,AskALiberal,2019-01-30


In [3]:
dataset['subreddit'].value_counts()

subreddit
loveafterlockup     227
socialanxiety       218
cringe              218
AnimalsBeingBros    216
timberwolves        209
                   ... 
canes                35
SSBM                 34
Anticonsumption      33
shieldbro            30
farcry               24
Name: count, Length: 483, dtype: int64

**Steps**:
+ Use Hugging Face model `SentenceTransformer('all-MiniLM-L6-v2')`
+ Encode each text into an embedding with method `encode()`
+ Group by subreddit and compute the mean embedding per subreddit
+ Cluster the subreddit embeddings using `KMeans`
+ Explore clusters and manually assign topic labels

In [4]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-V2')

dataset['embedding'] = dataset['text'].apply(lambda x: model.encode(x))

In [5]:
dataset['embedding'][0].shape

(384,)

In [6]:
subreddit_embeddings = (
    dataset.groupby('subreddit')['embedding']
    .apply(lambda x: sum(x) / len(x))
    .reset_index()
)

In [7]:
subreddit_embeddings

,subreddit,embedding
0,2meirl4meirl,"[-0.015225524, -0.003186024, 0.03313151, 0.006..."
1,49ers,"[-0.018552188, -0.0028821093, -0.010455885, -0..."
2,4PanelCringe,"[-0.023192884, 0.0039603934, 0.010494498, -0.0..."
3,90DayFiance,"[-0.019507552, -0.00010089574, 0.018924331, -0..."
4,90dayfianceuncensored,"[-0.016648285, -0.0016126882, 0.009415893, -0...."
...,...,...
478,worldpolitics,"[0.0021841251, 0.0019621311, 0.011987819, -0.0..."
479,yesyesyesyesno,"[-0.0083622085, 0.011073343, 0.018445078, -0.0..."
480,youseeingthisshit,"[-0.014335996, 0.014376306, 0.0155483335, -0.0..."
481,youtube,"[-0.023860248, -0.022737052, 0.0056497906, -0...."


In [8]:
subreddit_embeddings[subreddit_embeddings['subreddit']=='2meirl4meirl']['embedding'][0].shape

(384,)

In [9]:
n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
subreddit_embeddings['cluster'] = kmeans.fit_predict(list(subreddit_embeddings['embedding']))

In [10]:
def get_text_from_subreddit(df, subreddit):
    return df[df['subreddit']==subreddit]['text']

In [11]:
subreddit_embeddings

,subreddit,embedding,cluster
0,2meirl4meirl,"[-0.015225524, -0.003186024, 0.03313151, 0.006...",5
1,49ers,"[-0.018552188, -0.0028821093, -0.010455885, -0...",8
2,4PanelCringe,"[-0.023192884, 0.0039603934, 0.010494498, -0.0...",5
3,90DayFiance,"[-0.019507552, -0.00010089574, 0.018924331, -0...",9
4,90dayfianceuncensored,"[-0.016648285, -0.0016126882, 0.009415893, -0....",9
...,...,...,...
478,worldpolitics,"[0.0021841251, 0.0019621311, 0.011987819, -0.0...",3
479,yesyesyesyesno,"[-0.0083622085, 0.011073343, 0.018445078, -0.0...",5
480,youseeingthisshit,"[-0.014335996, 0.014376306, 0.0155483335, -0.0...",5
481,youtube,"[-0.023860248, -0.022737052, 0.0056497906, -0....",5


In [12]:
dataset

,text,labels,id,author,subreddit,created_utc,embedding
0,My favourite food is anything I didn't have to...,[27],eebbqej,Rockzilla78,KitchenConfidential,2019-01-18,"[0.01561645, 0.022650918, 0.031703655, 0.07407..."
1,"Now if he does off himself, everyone will thin...",[27],ed00q6i,cocaineluna,morbidquestions,2019-01-01,"[0.03288236, -0.05128799, -0.022228956, 0.0052..."
2,WHY THE FUCK IS BAYLESS ISOING,[2],eezlygj,minnesotagoat_,timberwolves,2019-01-26,"[-0.026340367, 0.019024402, 0.07136809, 0.0087..."
3,To make her feel threatened,[14],ed7ypvh,AlexNic1013,askwomenadvice,2019-01-04,"[0.001643833, 0.072182246, 0.06253768, 0.04545..."
4,Dirty Southern Wankers,[3],ed0bdzj,having_a_nap,london,2019-01-01,"[-0.049062364, -0.044063635, 0.01824677, -0.03..."
...,...,...,...,...,...,...,...
54258,Thanks. I was diagnosed with BP 1 after the ho...,[15],efeeasc,scaredyk4t,bipolar,2019-01-30,"[-0.05102062, -0.027362593, -0.0054752217, 0.0..."
54259,Well that makes sense.,[4],ef9c7s3,LX_Emergency,ABoringDystopia,2019-01-29,"[0.018713305, 0.017839583, 0.053923544, -0.051..."
54260,Daddy issues [NAME],[27],efbiugo,your_ex_girlfriend-,SquaredCircle,2019-01-30,"[-0.019132588, 0.012533258, -0.038491532, 0.02..."
54261,So glad I discovered that subreddit a couple m...,[0],efbvgp9,Paladin-Arda,AskALiberal,2019-01-30,"[-0.06342228, -0.09550535, -0.0056812265, 0.07..."


In [13]:
l = 0
for _, row in subreddit_embeddings.iterrows():
    l += len(get_text_from_subreddit(dataset, row['subreddit']))

print(l)

54263


In [14]:
pd.set_option('display.max_colwidth', None)

| **Cluster** | **Topic label** |
|---------|-------------|
| 0       | Humor       |
| 1       | Other       |
| 2       | Sport       |
| 3       | Politics       |
| 4       | Love and relationship       |
| 5       | Film and TV series       |
| 6       | Videogames       |

In [15]:
clusters = {
    'cluster': ['0', '1', '2', '3', '4', '5', '6'],
    'topic label': ['Humor', 'Other', 'Sport', 'Politics', 'Love and relationship', 'Film and Tv series', 'Videogames'],
    'Number of samples': [None] * 7
}

clusters = pd.DataFrame(clusters)

clusters

,cluster,topic label,Number of samples
0,0,Humor,None
1,1,Other,None
2,2,Sport,None
3,3,Politics,None
4,4,Love and relationship,None
5,5,Film and Tv series,None
6,6,Videogames,None


In [16]:
subreddit_embeddings['cluster'].value_counts()

cluster
5    106
3     89
7     68
4     65
9     47
2     40
6     40
8     22
0      5
1      1
Name: count, dtype: int64

In [17]:
# lenghts
def get_lenght(df, n_cluster):
    cluster = df[df['cluster']==n_cluster]
    lis = list(cluster['subreddit'])
    l = 0
    for s in lis:
        l += len(get_text_from_subreddit(dataset, s))

    return l 

In [18]:
sum = 0
for i in range(10):
    l = get_lenght(subreddit_embeddings, i)
    print(f"number of samples cluster {i} = {l}")
    sum += l 

print(sum)

number of samples cluster 0 = 439
number of samples cluster 1 = 45
number of samples cluster 2 = 4238
number of samples cluster 3 = 10273
number of samples cluster 4 = 9026
number of samples cluster 5 = 13307
number of samples cluster 6 = 3305
number of samples cluster 7 = 5839
number of samples cluster 8 = 2804
number of samples cluster 9 = 4987
54263


### Cluster 0 (Humor)

In [19]:
cluster0 = subreddit_embeddings[subreddit_embeddings['cluster']==0]
cluster0

,subreddit,embedding,cluster
112,Instagramreality,"[-0.009423519, 0.009711213, 0.023142347, -0.0014001647, -0.00069928664, -0.029505251, 0.04081976, -0.006224382, 0.005340592, -0.014167477, 0.001566375, -0.030661065, 0.0005357002, -0.018926734, -0.011095735, 0.007919993, 0.025833583, -0.0097108735, -0.027370814, 0.028470313, -0.01881666, 0.01684928, 0.025764681, 0.0107351495, -0.022724222, -0.010464401, 0.0054485807, -0.010136576, -0.00010675106, -0.0045077247, -0.027396929, 0.03188434, -0.01828086, 0.022229662, -0.020517435, 0.0043704975, 0.016051406, 0.036535516, -0.009056165, -0.015504201, -0.016905624, -0.024633463, -0.016651351, 0.013134284, -0.019643199, -0.015283076, 0.006353502, -0.023172943, 0.007735321, -0.041633613, -0.038090676, -0.044942033, -0.0127984695, -0.018595299, -0.0063101333, 0.021226032, 0.0040508197, -0.007214204, 0.031695, 0.011421279, 0.0032759365, 0.0350561, -0.006504772, 0.021229139, 0.02518246, -0.008555883, -0.0051475563, -0.014045771, -0.00477137, 0.010211583, 0.019594137, 0.0066176862, -0.013123977, 0.015422522, -0.0094084535, 0.010456675, 0.027201286, -0.019576633, 0.022996688, 0.035629876, 0.0011654315, -0.005282985, 0.024590576, 0.01313587, -0.00478079, -0.0011819986, -0.021363905, -0.053993173, -0.034840237, -0.0018340464, -0.037083484, -0.0011762105, 0.0050810776, -0.0071819737, 0.0017145508, -0.016650502, -0.015250044, 0.0016886136, -0.054664478, 0.056107268, ...]",0
119,Justfuckmyshitup,"[-0.028445255, 0.020124825, 0.018159807, -0.01889363, -0.010291366, -0.0233598, 0.03641206, -0.01823581, -0.002011181, -0.009708881, 0.00024799362, -0.009370487, -0.0008624026, -0.0014329958, -0.009760851, 0.01365203, 0.0055586784, -0.0065413453, -0.012341523, 0.014389235, -0.00960591, 0.022511987, 0.004617863, 0.0009209087, -0.014126093, 0.006409618, 0.01884187, 0.0081469435, -0.0054947357, -0.024437815, -0.008792562, 0.022309685, -0.015386082, 0.0041684653, -0.020729525, -0.014526561, 0.014603802, 0.042460423, 0.0080575785, -0.010795561, -0.008466319, -0.015917428, 0.0039134645, 0.009618145, -0.023050718, -0.0052705673, -0.0046686, -0.022830255, 0.026570382, -0.0055171154, -0.033912342, -0.04057475, 0.0013432605, -0.022055067, -0.0068200827, 0.020352798, 0.012213376, -0.00833372, 0.016933803, 0.018706389, 0.0015252109, 0.014753811, -0.015900455, 0.013424457, 0.015771333, -0.0146015305, -0.01037954, -0.030117022, 0.007183799, 0.011017408, 0.014972039, 0.006341818, -0.016595116, 0.008250077, -0.023909668, 0.0003717664, 0.013784275, -0.015869256, 0.019485, 0.023573475, 0.00052213424, -0.023213906, 0.007764485, 0.014112251, -0.009533212, 0.015211253, -0.01846821, -0.05141514, -0.024833005, 0.0063401717, -0.03348364, 0.0039197127, 0.009587608, -0.0037082992, -0.00676802, -0.0037339549, -0.005018573, -0.0047548427, -0.06821519, 0.06171023, ...]",0
180,RoastMe,"[-0.023977008, 0.005704335, 0.023990734, 0.0104857385, 0.0035585645, -0.038461212, 0.05999834, -0.008822876, 0.0020962676, -0.02349698, 0.00030247797, -0.047003254, 0.0044621816, 0.0072826254, -0.00369462, 0.00888722, 0.015431181, -0.012570888, -0.020640526, 0.012485693, -0.015021567, 0.035980314, 0.01150588, -0.005438355, -0.022300335, 0.02050426, 0.0015517788, 0.012893387, -0.015725002, -0.0068547106, -0.0030706255, 0.0036309764, -0.008186939, -0.005778624, -0.020557113, -0.0064146384, 0.009320637, 0.014014887, -0.0014028552, -0.0124380225, 0.0023785026, -0.015463007, -2.7425674e-05, 0.0030913204, -0.0027531541, 0.00093339774, -0.0072944164, -0.0070089055, 0.016537212, -0.02822569, -0.033815358, -0.033758193, 0.011261586, 7.111044e-05, -0.0020422079, 0.0090969615, 0.0027951063, 0.0044118436, 0.01681232, 0.0313192, -0.019822754, 0.020721681, -0.013594822, 0.016773997, 0.014778312, 0.0026015248, -0.008211727, -0.0051552663, 0.003227567, 0.030924164, 0.005631908, -0.00046669453, -0.015756505, 0.0038585863, -0.017487302, 0.0042773234, 0.024494272, -0.036877684, 0.03976713, 0.044441212, -0.01910028, -0.008830195, -0.0015914576, 0.009190792, -0.024277445, -

In [20]:
# cluster 5 -> cluster 0
subreddit_embeddings.loc[subreddit_embeddings['cluster']==5, 'cluster'] = 0

# cluster 1 -> cluster 0 
subreddit_embeddings.loc[subreddit_embeddings['cluster']==1, 'cluster'] = 0

In [21]:
subreddit_embeddings[subreddit_embeddings['cluster']==0]

,subreddit,embedding,cluster
0,2meirl4meirl,"[-0.015225524, -0.003186024, 0.03313151, 0.006350386, 0.024410237, -0.025254453, 0.04390977, -0.0058279918, 0.022215879, -0.02113416, -0.008272042, -0.017277224, 0.0005259701, 0.0146532515, -0.00018366391, 0.0065619093, 0.015795756, -0.0196112, -0.03444035, 0.01683993, -0.022603214, 0.025492871, -0.0027485809, 0.0030058245, -0.017045945, 0.017293913, -0.0053343, -0.00092097145, -0.014344458, -0.004288831, -0.013948282, 0.027729714, -0.00624809, 0.0010195352, -0.0011348142, -0.006760229, -0.0039269924, -0.005572145, 0.004685422, 0.0019570033, 0.0055032135, 0.0016195015, 0.012987115, 0.0010843549, 0.004612709, -0.004861469, -0.0102725625, -0.015235719, 0.036755502, -0.012383166, -0.023019752, -0.016407376, -0.014417428, -0.0028915976, 0.017426072, 0.02310031, 0.005130199, 0.010803099, 0.024310114, 0.005927755, -0.013256294, 0.015327123, -0.0073854825, 0.0061507258, 0.02964968, 0.007495445, -0.009338666, 0.0022637343, -0.010387419, 0.028194552, -0.005761978, 0.008700418, -0.009146753, 0.020950273, -0.013040407, 0.007349688, 0.009512057, -0.04226785, 0.018402874, 0.032723792, -0.00872738, -0.008868415, -0.0090967035, 0.0004565285, -0.013747484, -0.009730945, 0.013441699, -0.014010522, 0.005549174, 0.005738937, -0.026287165, -0.0064369226, 0.0094893575, 0.013410227, -0.016965017, -0.022709789, -0.020837422, 0.01089403, -0.0650929, 0.069009155, ...]",0
2,4PanelCringe,"[-0.023192884, 0.0039603934, 0.010494498, -0.015115168, 0.006836462, -0.0257574, 0.062217627, -0.003969516, 0.019752953, -0.013393847, -0.009315065, -0.014945045, 0.0043270197, 0.007525347, -0.03317322, -0.0010646626, 0.020012263, -0.011337729, -0.012260013, -0.0053116763, -0.011579572, 0.03377392, 0.018386476, -0.008251519, -0.015098509, 0.007776889, 0.0002013597, 0.0042218273, -0.012523291, -0.0047490625, -0.011362211, 0.03457132, 0.005539218, 0.012535796, -0.020651136, -0.013078647, 0.01133792, 0.022744292, 0.004049748, -0.0032217214, 0.0057809227, -0.028601117, 0.0044739638, 0.0004291405, -0.0010671394, 0.012152756, -0.011778911, -0.013840539, 0.026533436, -0.01406982, -0.029432291, -0.030920757, -0.003413601, -0.00934403, 0.0029896398, 0.014994485, -0.0032579044, 0.0051108813, 0.024964232, 0.018867476, -0.005563573, 0.013376458, -0.010095878, 0.007834922, 0.023456518, -0.020910878, -0.0076802387, 0.000558006, -0.01619879, 0.022920784, 0.009963119, 0.015570447, -0.00830459, 0.027910661, -0.015695438, 0.0068163234, 0.005929091, -0.024902588, 0.01146984, 0.022460744, -0.010102586, -0.020960169, -0.0054418645, -0.0035604178, -0.004414795, -0.0075443047, 0.0075835157, -0.027635863, -0.00678159, -0.002211597, -0.029570624, -0.009576335, 0.026818966, 0.013858904, -0.006839696, -0.022065274, -0.017180715, 0.0027139594, -0.061458662, 0.07802418, ...]",0
5,ABCDesis,"[-0.007688805, 4.959004e-05, 0.0069934307, -0.0055983183, 0.0032650512, -0.011741787, 0.03038204, -0.014807505, 0.007947136, -0.012076037, 0.003230898, -0.01285105, 0.0019093927, 0.0009877848, 0.0014984016, -0.0013246085, 0.015998492, -0.029927028, -0.021650009, 0.0040712664, -0.023225667, 0.00967296, 0.013818662, 0.006238326, -0.0034466884, 0.0011465409, 0.006733888, 0.0024624155, -0.013791796, 0.006583616, -0.0018984022, 0.03268112, -0.007853035, 0.010652667, -0.015161123, -0.00362744, 0.008721226, 0.0186222, 0.011017967, -0.008408868, 0.012091029, -0.0031401874, 0.015855331, -0.004799152, -0.0063970066, -0.001979191, -0.018134931, -0.018342925, 0.016507896, -0.014161355, -0.009721644, -0.010209368, -0.0039509293, -0.005315116, 0.0035914986, 0.007120031, -0.0019474333, 0.009791052, 0.005880823, 0.013425245, -0.009770667, 0.013574024, -0.019821819, 0.017688211, 0.01876128, 0.00039993125, -0.014399096, 0.012133308, -0.00025717064, 0.010213571, 0.0066920137, 0.012365317, -0.0027927323, 0.026027655, -0.008657194, -0.0022087006, 0.012339492, -0.01857518, 0.016794272, 0.018440153, -0.013866387, 0.0075395554, -0.0011610696, 0.00012739225, -0.0007821403, -0.01607039, 0.0029

In [22]:
humor = list(subreddit_embeddings[subreddit_embeddings['cluster']==0]['subreddit'])
print(humor)

['2meirl4meirl', '4PanelCringe', 'ABCDesis', 'ATBGE', 'AnimalsBeingBros', 'AnimalsBeingJerks', 'BetterEveryLoop', 'Blackfellas', 'BreadTube', 'Cardinals', 'ComedyCemetery', 'DiWHY', 'Documentaries', 'ExpectationVsReality', 'FellowKids', 'FrankOcean', 'FunnyandSad', 'HadToHurt', 'HighQualityGifs', 'HumansBeingBros', 'IASIP', 'IDontWorkHereLady', 'IdiotsInCars', 'Instagramreality', 'Jokes', 'Justfuckmyshitup', 'KidsAreFuckingStupid', 'MaliciousCompliance', 'Military', 'My600lbLife', 'ProRevenge', 'Roadcam', 'RoastMe', 'SubredditDrama', 'SubredditSimulator', 'SweatyPalms', 'TIHI', 'TheSimpsons', 'Tinder', 'ToolBand', 'TrollXChromosomes', 'TwoSentenceHorror', 'Unexpected', 'WatchPeopleDieInside', 'WeWantPlates', 'Wellthatsucks', 'Whatcouldgowrong', 'WhitePeopleTwitter', 'WhyWereTheyFilming', 'WouldYouRather', 'antiMLM', 'awfuleverything', 'barstoolsports', 'bestoflegaladvice', 'bodybuilding', 'circlejerk', 'cringe', 'cringepics', 'dadjokes', 'delusionalartists', 'disneyvacation', 'dogswith

In [23]:
l = 0
for s in humor:
    l += len(get_text_from_subreddit(dataset, s))

print(l)

13791


In [24]:
clusters.loc[clusters['cluster']=='0', 'Number of samples'] = l

In [25]:
clusters

,cluster,topic label,Number of samples
0,0,Humor,13791
1,1,Other,None
2,2,Sport,None
3,3,Politics,None
4,4,Love and relationship,None
5,5,Film and Tv series,None
6,6,Videogames,None


In [26]:
subreddit_embeddings['cluster'].value_counts().sort_index()

cluster
0    112
2     40
3     89
4     65
6     40
7     68
8     22
9     47
Name: count, dtype: int64

### Cluster 1 (Other)

In [27]:
subreddit_embeddings[subreddit_embeddings['cluster']==1]

,subreddit,embedding,cluster


In [28]:
# cluster 7 -> cluster 1
subreddit_embeddings.loc[subreddit_embeddings['cluster']==7, 'cluster'] = 1

In [29]:
subreddit_embeddings[subreddit_embeddings['cluster']==1]

,subreddit,embedding,cluster
11,AirForce,"[-0.012958019, -0.0069180746, 0.018451672, 0.0009916542, 0.017646557, -0.021794278, 0.0010664454, -0.00628202, -0.0035156063, 0.002326863, 0.0066808467, -0.0015685555, -0.00061749044, 0.020783747, -0.010334581, 0.0019829178, 0.017770173, -0.0187666, -0.017817834, 0.009877738, -0.0061971126, 0.009666472, 2.6805386e-05, 0.0049631256, -0.0065777134, 0.008989308, -0.008255654, 0.020244556, -0.011506865, -0.011309002, -0.02181735, 0.025937589, 0.015929837, 0.004559538, 0.011831123, 0.010093267, 0.017419208, 0.004937216, 0.0066709733, -0.0073600477, 0.0012438388, -0.013577104, 0.014145654, 0.016368054, -0.00073677534, -0.002814681, -0.0025655474, -0.023928039, 0.02532541, 0.0024689927, 0.008815608, -0.03115194, -0.010804708, -0.0035275903, -0.00023483431, 0.012075771, 0.003675383, -0.0006788646, 0.0031562864, -0.0026064438, -0.010218615, 0.003150089, -0.027151247, 0.017197454, 0.011475104, 0.0028776454, -0.0061341533, 0.008061484, 0.006613307, 0.0020716474, -0.006109671, 0.0026271741, 0.00707731, 0.028696937, 0.0027800757, 0.023865, 0.0016895417, -0.014934791, 0.040216904, 0.0046215444, -0.0055862824, -0.006989507, -0.0061541214, -0.005099175, -0.0023583155, -0.024220387, 0.0042667408, -0.0058325212, 0.015358214, -0.0025558977, 0.0030779743, 0.00018593844, 0.025883332, 0.015202893, -0.03741629, -0.013019238, -0.023500266, 0.0124651315, -0.050032042, 0.04094905, ...]",1
16,Anticonsumption,"[-0.022477584, 0.0036739495, 0.012291948, -0.0072236676, 0.003764357, -0.025016276, 0.012985981, -0.006268739, -0.0045521585, -0.0010381355, -0.0043603056, 0.039055053, 0.0070561073, 0.023062753, -0.028608723, -0.005298428, 0.011578347, -0.024696179, -0.0175849, -0.0025409437, -0.01387831, 0.008160904, 0.018801572, -0.009282779, -0.018044349, 0.016798427, -0.0061958786, 0.006005861, -0.022681544, -0.01531192, -0.004703216, 0.027922247, -0.0034550352, -0.011018619, -0.0052073374, 0.0068552233, 0.025496772, 0.0021923322, 0.008330526, 0.0013591166, -0.0054605613, 0.002365284, 0.016168624, 0.0045748395, -0.019632215, 0.0057937154, -0.00096087076, -0.02171194, 0.029842554, -0.0051424857, -0.0074149696, -0.0129286535, -0.017604686, -0.034504913, -0.010748338, 0.015895817, -0.012692672, -0.007863778, 0.0069235004, -0.0083334, 0.020346304, 0.002337122, -0.008329173, 0.02011764, 0.026229225, -0.0042799083, -0.015473554, 0.01838112, -0.012235214, 0.005112015, -0.013204487, 0.024074899, -0.0027325628, 0.033518612, -0.011821434, 0.009654809, 0.02201902, -0.03761056, -0.00015438865, 0.019461904, -0.019313723, 0.0040772, -0.0009764564, -0.000840242, -0.0047618756, -0.014509122, -0.0010756779, -0.026974956, -0.004269645, -0.009806287, -0.011158184, -0.0021346598, 0.023027645, 0.014994702, -0.018984042, -0.005916492, -0.013762516, -0.0022816912, -0.05634011, 0.04268157, ...]",1
25,Austin,"[0.0060520344, 0.006788532, 0.025984237, 0.006586112, 0.0065978845, -0.01956996, 0.006377114, -0.02494813, -0.008559187, -0.0031299682, 0.017262133, 0.010938329, 0.013373616, 0.0051856874, -0.010141937, -0.0072025363, 0.014725178, -0.029835658, -0.011648055, 0.002375717, -0.010861316, 0.017871477, -0.0020556997, 0.014115174, -0.008388866, 0.018532101, -0.013822462, 0.0129696885, -0.018526742, -0.00498112, -0.011858051, 0.040165477, 0.0031268774, 0.009252624, 0.012718369, 0.00021709027, 0.021509167, -0.0071818843, 0.015974987, 0.00066311815, -0.0013249946, -0.016884804, 0.008115332, 0.009239311, -0.012531657, 0.0055770637, -0.012001957, -0.021469133, 0.035049874, -0.0141382795, 0.011365543, -0.010761786, 0.0026272328, -0.0086800605, -0.0076787826, 0.0076072263, -0.0074991807, -0.008125783, 0.010382127, 0.010737461, 0.012908555, 0.011166551, -0.029346874, 0.020506833, 0.012987025, -0.003683116, -0.02837827, -0.002021634, 0.0119922105, 0.0014771539, 0.013632246, 0.006393906, 0.010604181, 0.020190341, 0.005368974, 0.011721891, 0.017205596, -0.017075635, 0.021159438, 0.0015173693, -0.013958084, -0.019875485, -0.009511497, 0.0051114103, -0.0043

In [30]:
other = list(subreddit_embeddings[subreddit_embeddings['cluster']==1]['subreddit'])
print(other)

['AirForce', 'Anticonsumption', 'Austin', 'Calgary', 'ChoosingBeggars', 'Connecticut', 'Dallas', 'Denver', 'Detroit', 'Edmonton', 'Flipping', 'GifRecipes', 'InstacartShoppers', 'KitchenConfidential', 'LifeProTips', 'LosAngeles', 'NewOrleans', 'Portland', 'ProtectAndServe', 'QuadrigaCX', 'SaltLakeCity', 'SeattleWA', 'StLouis', 'TalesFromRetail', 'TalesFromYourServer', 'Target', 'Teachers', 'Truckers', 'askcarsales', 'australia', 'benzodiazepines', 'brisbane', 'britishproblems', 'btc', 'canadients', 'chicago', 'darknet', 'doordash', 'ethtrader', 'houston', 'ireland', 'jobs', 'juul', 'korea', 'legaladvice', 'london', 'melbourne', 'minnesota', 'nashville', 'newjersey', 'newzealand', 'nursing', 'nyc', 'perth', 'phoenix', 'pittsburgh', 'sanfrancisco', 'southafrica', 'starbucks', 'sydney', 'texas', 'toronto', 'uberdrivers', 'vancouver', 'vegan', 'walmart', 'washingtondc', 'weed']


In [31]:
l = 0
for s in other:
    l += len(get_text_from_subreddit(dataset, s))

print(l)

5839


In [32]:
clusters.loc[clusters['cluster']=='1', 'Number of samples'] = l

In [33]:
clusters

,cluster,topic label,Number of samples
0,0,Humor,13791
1,1,Other,5839
2,2,Sport,None
3,3,Politics,None
4,4,Love and relationship,None
5,5,Film and Tv series,None
6,6,Videogames,None


### Cluster 2 (Sport)

In [34]:
subreddit_embeddings[subreddit_embeddings['cluster']==2] #sport

,subreddit,embedding,cluster
7,AFL,"[-0.012297828, -0.00060421904, -0.008414087, -0.018882267, 0.016993154, -0.0010358654, 0.04344437, -0.005131242, -0.008717113, 0.001504226, -0.033724684, -0.016960237, 0.00041312055, 0.020144595, -0.009974679, 0.009131653, 0.0016041128, -0.027630936, -0.0006431617, -0.010434832, -0.024380283, 0.02390933, -0.018400509, 0.0070949765, -0.014729497, 0.0056430106, -0.007975143, 0.027667882, -0.012850241, -0.017531455, -0.0062555773, 0.04199416, 0.018740445, 0.008254486, -0.018111434, -0.019052532, -0.010761204, 0.0039838892, -0.0077459337, -0.008689191, 0.02856639, -0.027199782, -0.00419254, 0.01986054, -0.01774702, -5.9711747e-05, -0.0018267494, -0.024870379, 0.009796092, 0.026341593, -0.02290974, -0.011651708, 0.0042538596, -0.016690113, 0.007830553, 0.034959678, -0.013920025, -0.0059651053, 0.0041742236, -0.007782833, 0.0018726853, 0.011532291, -0.05210236, -0.00023045059, 0.006417493, -0.020902317, -0.025449455, 0.022250658, 0.010743278, 0.013003652, 0.024687778, 0.0032347993, 0.011580168, -0.0060339463, 0.0012071735, 0.018033093, 0.004475032, -0.015810842, 0.03276503, 0.010764859, -0.0062289005, -0.023200057, -0.02276081, -0.00765531, -0.0023535984, 0.0047808033, -0.0076981834, -0.01919262, 0.008049161, -0.003773391, -0.019369952, -0.017437717, 0.031862304, 0.012674909, -0.0066905185, 0.010713278, -0.009491305, 0.007610279, -0.07093891, 0.07195311, ...]",2
24,AtlantaHawks,"[-0.0315351, 0.0014153741, -0.01976336, -0.038198095, 0.005658214, -0.019273447, 0.08157149, 0.00065174344, 0.017463373, -0.015803155, -0.041174695, -0.030677766, 0.007104755, 0.014713469, -0.0017132455, 0.014855582, 0.019448437, 0.013877335, -0.007706429, -0.012939681, -0.018769719, 0.02788166, -0.0033413805, -0.0032936125, -0.008951711, 0.004672571, 0.007185096, 0.017069176, -0.008688326, -0.031370047, -0.0009888352, 0.025514042, 0.014755453, 0.011058745, -0.027644863, -0.018373372, -0.03143744, 0.039328463, -0.00850512, -0.016942488, 0.03155596, -0.036294222, -0.0007055989, 0.024533655, -0.04051224, 8.5347034e-05, -0.014421494, -0.024255754, 0.03312818, 0.011838611, -0.047509953, -0.02074461, 0.013761496, 0.0064141206, 0.0066576516, 0.03934354, -0.0041241543, 0.0020466854, 0.01885187, 0.019672986, -0.0010653331, 0.009582471, -0.02474562, -0.016847542, 0.0061981296, -0.028462667, -0.007227445, 0.00048526478, 0.010221528, 0.030476172, 0.037682362, 0.025957564, -0.010543519, -0.022674538, -0.003867771, 0.022691103, -0.015963554, -0.031762287, 0.017123848, 0.02312578, -0.009901335, -0.016753137, -0.023263026, 0.004265947, -0.029990643, 0.03203627, -0.020615406, -0.0015990398, 0.009266502, -0.012815217, -0.013546166, -0.025056073, 0.033745747, 0.0015148405, -0.0016077937, 0.011543222, -0.010542994, -0.016444752, -0.08104232, 0.07143829, ...]",2
28,Barca,"[-0.025670474, 0.01977418, -0.01190819, -0.016351767, 0.0156090455, 0.0042018015, 0.069448605, 0.0008320357, 0.008617658, -0.017602244, -0.025469061, -0.01659686, 0.011655771, 0.020454764, -0.010726174, 0.010855266, -0.008453026, -0.009352836, -0.0042385855, -0.019094465, 0.003905233, 0.029229602, -0.010185215, 0.0015092741, -0.008361664, 0.015419983, -0.011307665, -0.00075201225, -0.0012750279, -0.043132674, 0.015588519, 0.034438457, 0.018698756, 0.01290723, -0.02843792, -0.015712414, -0.021037295, 0.020707333, -0.012740027, -0.019587459, 0.024416378, -0.03810207, -0.016648458, 0.022638189, -0.01706956, 0.0015465618, 0.009305166, -0.026018877, 0.019060668, 0.017649144, -0.04465892, -0.009309109, 0.0044891103, 0.005549996, -0.0046303235, 0.027086545, 0.0031662583, -0.009357456, 0.009259983, 0.037095785, -0.016005792, 0.022116352, -0.04561835, -0.0048254086, 0.0021618775, -0.046244983, -0.012419529, 0.0032831447, 0.007754066, 0.05484307, 0.022164145, 0.014832152, 0.0062472397, -0.018652376, -0.010860168, 0.004668108, -0.030769523, -0.024524374, 0.022709655, 0.012929849, -0.009054198, -0.028219758, -0.032289892, -0.008197346, 0.0027919444, 0.03463212, -0.012433803, -0.01707175

In [35]:
# cluster 8 -> cluster 2
subreddit_embeddings.loc[subreddit_embeddings['cluster']==8, 'cluster'] = 2

In [36]:
subreddit_embeddings[subreddit_embeddings['cluster']==2] #sport

,subreddit,embedding,cluster
1,49ers,"[-0.018552188, -0.0028821093, -0.010455885, -0.031844795, 0.010581031, -0.005172225, 0.046305984, 0.011533315, 0.01129894, -0.013584505, -0.04259195, 0.0033355802, 0.009804972, 0.010599564, -0.0006873135, 0.008815442, 0.008406114, 0.00026762337, -0.0016090338, -0.014573603, -0.01664769, 0.032211084, -0.016571626, -0.004707403, 0.0015726627, -0.012411625, -0.005187421, 0.02579788, -0.02105614, -0.017291166, 0.0055347923, 0.031248076, 0.021179467, 0.000675637, -0.023365647, -0.010130943, -0.024683941, 0.027590336, 0.00096075144, -0.00077199616, 0.014498017, -0.031289928, -0.0017865194, 0.020937946, -0.040670794, -0.011148991, -0.014191534, -0.022238376, 0.016479896, 0.005604676, -0.026643623, -0.007814203, 0.012288588, 0.0015518678, 0.016842334, 0.04356018, -0.018324012, -0.004788586, -0.005676649, 0.021088494, -0.0073766876, 0.014238733, -0.026548555, -0.009252305, -0.0009091181, -0.0142886685, -0.00739615, 0.0075662225, -0.004795422, 0.020544996, 0.031024555, 0.0199509, -0.0011819891, -0.006068955, 0.010829666, 0.011118542, -0.00401216, -0.009677233, 0.031046646, 0.00692493, -0.012846708, -0.030788206, -0.02258014, -0.0055785608, -0.021892007, 0.031249726, -0.013291126, -0.020846916, 0.013081853, 0.00636993, -0.010408722, -0.040105782, 0.030874673, -0.0016292842, 0.0044602375, 0.00819574, 0.0022628978, -0.008834058, -0.074583605, 0.0638592, ...]",2
7,AFL,"[-0.012297828, -0.00060421904, -0.008414087, -0.018882267, 0.016993154, -0.0010358654, 0.04344437, -0.005131242, -0.008717113, 0.001504226, -0.033724684, -0.016960237, 0.00041312055, 0.020144595, -0.009974679, 0.009131653, 0.0016041128, -0.027630936, -0.0006431617, -0.010434832, -0.024380283, 0.02390933, -0.018400509, 0.0070949765, -0.014729497, 0.0056430106, -0.007975143, 0.027667882, -0.012850241, -0.017531455, -0.0062555773, 0.04199416, 0.018740445, 0.008254486, -0.018111434, -0.019052532, -0.010761204, 0.0039838892, -0.0077459337, -0.008689191, 0.02856639, -0.027199782, -0.00419254, 0.01986054, -0.01774702, -5.9711747e-05, -0.0018267494, -0.024870379, 0.009796092, 0.026341593, -0.02290974, -0.011651708, 0.0042538596, -0.016690113, 0.007830553, 0.034959678, -0.013920025, -0.0059651053, 0.0041742236, -0.007782833, 0.0018726853, 0.011532291, -0.05210236, -0.00023045059, 0.006417493, -0.020902317, -0.025449455, 0.022250658, 0.010743278, 0.013003652, 0.024687778, 0.0032347993, 0.011580168, -0.0060339463, 0.0012071735, 0.018033093, 0.004475032, -0.015810842, 0.03276503, 0.010764859, -0.0062289005, -0.023200057, -0.02276081, -0.00765531, -0.0023535984, 0.0047808033, -0.0076981834, -0.01919262, 0.008049161, -0.003773391, -0.019369952, -0.017437717, 0.031862304, 0.012674909, -0.0066905185, 0.010713278, -0.009491305, 0.007610279, -0.07093891, 0.07195311, ...]",2
24,AtlantaHawks,"[-0.0315351, 0.0014153741, -0.01976336, -0.038198095, 0.005658214, -0.019273447, 0.08157149, 0.00065174344, 0.017463373, -0.015803155, -0.041174695, -0.030677766, 0.007104755, 0.014713469, -0.0017132455, 0.014855582, 0.019448437, 0.013877335, -0.007706429, -0.012939681, -0.018769719, 0.02788166, -0.0033413805, -0.0032936125, -0.008951711, 0.004672571, 0.007185096, 0.017069176, -0.008688326, -0.031370047, -0.0009888352, 0.025514042, 0.014755453, 0.011058745, -0.027644863, -0.018373372, -0.03143744, 0.039328463, -0.00850512, -0.016942488, 0.03155596, -0.036294222, -0.0007055989, 0.024533655, -0.04051224, 8.5347034e-05, -0.014421494, -0.024255754, 0.03312818, 0.011838611, -0.047509953, -0.02074461, 0.013761496, 0.0064141206, 0.0066576516, 0.03934354, -0.0041241543, 0.0020466854, 0.01885187, 0.019672986, -0.0010653331, 0.009582471, -0.02474562, -0.016847542, 0.0061981296, -0.028462667, -0.007227445, 0.00048526478, 0.010221528, 0.030476172, 0.037682362, 0.025957564, -0.010543519, -0.022674538, -0.003867771, 0.022691103, -0.015963554, -0.031762287, 0.017123848, 0.02312578, -0.009901335, -0.016753137, -0.023263026, 0.004265947, -0.029990643, 0.03203627, -0.020615406, -0.001599

In [37]:
sport = list(subreddit_embeddings[subreddit_embeddings['cluster']==2]['subreddit'])
print(sport)

['49ers', 'AFL', 'AtlantaHawks', 'Barca', 'Boxing', 'Braves', 'Browns', 'CFB', 'CollegeBasketball', 'DenverBroncos', 'DetroitPistons', 'Dodgers', 'EdmontonOilers', 'GoNets', 'GreenBayPackers', 'Gunners', 'Habs', 'LigaMX', 'MLS', 'MMA', 'Mavericks', 'MkeBucks', 'NYGiants', 'NYKnicks', 'NYYankees', 'NewYorkIslanders', 'NewYorkMets', 'OrlandoMagic', 'OttawaSenators', 'Padres', 'ScottishFootball', 'TeamSolomid', 'canes', 'canucks', 'chelseafc', 'chicagobulls', 'denvernuggets', 'detroitlions', 'devils', 'fantasyfootball', 'goldenknights', 'hawks', 'hockey', 'leafs', 'minnesotavikings', 'nba', 'nrl', 'nyjets', 'penguins', 'realmadrid', 'ripcity', 'rugbyunion', 'sabres', 'soccer', 'sports', 'sportsbook', 'steelers', 'tennis', 'timberwolves', 'torontoraptors', 'wildhockey', 'winnipegjets']


In [38]:
l = 0
for s in sport:
    l += len(get_text_from_subreddit(dataset, s))

print(l)

7042


In [39]:
clusters.loc[clusters['cluster']=='2', 'Number of samples'] = l

In [40]:
clusters

,cluster,topic label,Number of samples
0,0,Humor,13791
1,1,Other,5839
2,2,Sport,7042
3,3,Politics,None
4,4,Love and relationship,None
5,5,Film and Tv series,None
6,6,Videogames,None


### Cluster 3 (Politics)

In [41]:
subreddit_embeddings[subreddit_embeddings['cluster']==3] #politics

,subreddit,embedding,cluster
6,ABoringDystopia,"[0.0072658556, 0.015259034, 0.014032003, -0.0026088099, 0.019907791, -0.008377345, 0.02459876, -0.01804409, 0.0062713344, 0.0061978973, 0.013386116, 0.022665164, 0.010316749, -0.0015940014, 0.0025742946, -3.6364767e-05, 0.013361589, -0.034753103, -0.020867964, 0.016733149, -0.016676554, 0.020626614, 0.015496618, 0.013178944, -0.005369803, 0.006652038, -0.010248697, 0.008933416, -0.007739651, 0.015044638, -0.005429787, 0.017159726, 0.0055317404, 0.012057486, -0.0071740556, 0.018146131, 0.042175956, 0.005336225, 0.017161693, -0.0073676393, 0.010539883, -0.026278099, 0.01450582, 0.0039401283, 0.0010339229, 0.0020493097, -0.010051464, -0.017349267, 0.018506305, -0.039857473, 0.022366753, -0.02585911, -0.0061835772, -0.016552188, 0.0049012583, -0.0038619016, 0.0050200922, 0.0028201798, 0.0029595052, 0.018895317, -0.024579369, 0.012161625, -0.014825955, 0.013697609, 0.0310916, 0.009155423, 0.0064553893, 0.010493841, -0.0008182162, 0.018587219, -0.0008696722, 0.0076991287, 0.01170394, 0.021246895, -0.016088368, -0.006009507, 0.005809683, -0.0033061919, 0.02356801, 0.008883669, 0.0048602843, -0.017486397, -0.010509205, 0.013380062, 0.0037136371, -0.015719835, 0.00037063443, -0.025682347, 0.024979686, 0.01049575, -0.027280547, -0.00583275, 0.018100834, 0.001029627, -0.010074375, -0.015081249, -0.016657013, 0.014293071, -0.05206539, 0.044596367, ...]",3
10,AdviceAnimals,"[-0.0051695043, 0.006849251, 0.013661254, -0.0010641126, 0.015339205, -0.015010577, 0.034139685, -0.0055909953, 0.007780489, -0.014037821, 0.0055802613, 0.0060521825, 0.0063170874, 0.0055033355, 0.0002718995, 0.001892615, 0.013668114, -0.030221704, -0.01733655, 0.015732726, -0.014114725, 0.011135993, 0.013905521, 0.0036650526, -0.008508468, 0.0032762887, -0.013016727, -0.00039120743, -0.016061142, 0.013217792, -0.0054501104, 0.03323616, 0.0003281428, 0.013338425, -0.008042652, -0.0048774695, -5.864447e-05, 0.013929911, 0.0057244855, -0.012236214, 0.011825076, -0.014381618, 0.007467672, 0.0048342803, -0.0060412977, -0.0010126518, -0.0030128935, -0.003756019, 0.022555742, -0.014385543, -0.014289804, -0.011750639, -0.0034092658, 0.0010214138, 0.0025244977, -0.0037446509, -0.005667478, 0.010908436, 0.010459744, 0.01610493, -0.011091395, 0.010760288, -0.0124276215, 0.017864978, 0.018195393, 0.019413866, -0.009572754, 0.0045994017, -0.01776303, 0.029920887, -0.0007182735, 0.012463987, 0.0042761043, 0.031219553, -0.007880627, -0.0045077195, 0.010293487, -0.012239668, 0.024942437, 0.024614526, -0.01164216, -0.0008971367, -0.002635425, 0.0069949366, 0.004741714, -0.014646423, -0.004620522, -0.018490307, 0.011394632, 0.0069173994, -0.02089605, -0.017878914, 0.036224656, 0.006270936, -0.0076630735, -0.014028824, -0.013569522, 0.02009956, -0.056344427, 0.048553184, ...]",3
12,Anarchism,"[0.0038602331, -0.003896529, 0.003364831, -0.0018519738, 0.012476095, -0.007551637, 0.020387134, -0.019857721, -0.008352634, 0.0012901434, 0.019419797, 0.013532631, 0.009490641, 0.008527075, -0.015732242, 0.012130169, 0.007298019, -0.037656333, -0.022869932, 0.019773219, -0.012847048, 0.015796239, 0.021516085, 0.0148632135, -0.013224804, 0.0023302343, -0.0030405107, 0.0065966006, -0.008375858, 0.010978446, -0.010173895, 0.020827103, -0.006934311, 0.018017512, 0.001334022, -0.0062628067, 0.03593125, 0.015379562, 0.01354877, -0.015654838, 0.0126664275, -0.016770411, 0.009698516, 0.0023081747, -0.010528195, 0.014360323, -0.017750887, -0.008833526, 0.014734573, -0.036007695, -0.012384122, -0.015032934, -0.00567076, -0.00013682712, 1.2298597e-06, -0.0026430006, 0.0012919146, 0.006642919, 0.0127333235, 0.0017261421, 0.0010407649, 0.0038328734, -0.020758914, 0.017906863, 0.028916392, -0.0012309994, -0.013489099, 0.014971932, -0.021593647, 0.012858097, 0.008065887, 0.0067839916, 0.019561654, 0.03090938, -0.015500818, -0.012622542, 0.012600643, -0.013873089, 0.017056387, 0.007850643, 0.0005133495, 0.006534274, -0.0016424981, 0.0007394012, 0.0075867306, -0.01095

In [42]:
politics = list(subreddit_embeddings[subreddit_embeddings['cluster']==3]['subreddit'])
print(politics)

['ABoringDystopia', 'AdviceAnimals', 'Anarchism', 'Anarcho_Capitalism', 'AskALiberal', 'AskFeminists', 'AskThe_Donald', 'COMPLETEANARCHY', 'CanadaPolitics', 'CapitalismVSocialism', 'Catholicism', 'China', 'Christianity', 'Conservative', 'CryptoCurrency', 'DebateAnAtheist', 'ENLIGHTENEDCENTRISM', 'Enough_Sanders_Spam', 'Futurology', 'GCdebatesQT', 'GenderCritical', 'IncelTears', 'JordanPeterson', 'Judaism', 'KotakuInAction', 'LabourUK', 'LateStageCapitalism', 'Libertarian', 'MakingaMurderer', 'MensRights', 'MorbidReality', 'MurderedByWords', 'OutOfTheLoop', 'PoliticalDiscussion', 'PoliticalHumor', 'PurplePillDebate', 'SandersForPresident', 'Scotland', 'SelfAwarewolves', 'ShitPoliticsSays', 'Shitstatistssay', 'The_Donald', 'The_Mueller', 'TopMindsOfReddit', 'TrueReddit', 'UpliftingNews', 'VoteBlue', 'WayOfTheBern', 'alberta', 'antinatalism', 'atheism', 'aznidentity', 'badunitedkingdom', 'belgium', 'bestof', 'brexit', 'canada', 'changemyview', 'collapse', 'conspiracy', 'environment', 'exc

In [43]:
l = 0
for s in politics:
    l += len(get_text_from_subreddit(dataset, s))

print(l)

10273


In [44]:
clusters.loc[clusters['cluster']=='3', 'Number of samples'] = l

In [45]:
clusters

,cluster,topic label,Number of samples
0,0,Humor,13791
1,1,Other,5839
2,2,Sport,7042
3,3,Politics,10273
4,4,Love and relationship,None
5,5,Film and Tv series,None
6,6,Videogames,None


### Cluster 4 (Love and relationship)

In [46]:
subreddit_embeddings[subreddit_embeddings['cluster']==4] # love and relationship

,subreddit,embedding,cluster
9,Advice,"[-0.007617102, -0.006089481, 0.023801913, -0.0049976697, 0.006722086, -0.009879444, 0.0027762912, -0.013150741, -0.0022804427, -0.025250545, 0.003895956, -0.008447441, -0.014658925, 0.017414179, 0.00083230727, 0.011905843, 0.020765819, -0.017536398, -0.025998984, 0.0098259365, 0.006029329, -0.000421811, 0.0010565891, -0.0071402416, -0.011854249, 0.01179078, -0.0069694854, -0.002212876, -0.009973843, 0.005469132, -0.013630164, 0.023226213, -0.0039365073, -0.0019264787, 0.0012191379, -0.0013369834, 0.00090062193, 0.008878565, 0.00906951, -0.0061895913, 0.011221791, 0.01499977, 0.016571833, -0.002558864, -0.014552178, 0.011065244, -0.0067599528, -0.017204944, 0.02439022, -0.013878089, -0.013594297, -0.01876017, -0.016748287, 0.004117605, 0.016464056, 0.010404959, -0.001767, 0.0034639859, 0.013612827, 0.0014033313, -0.014266689, 0.018927049, -0.019044824, 0.01787677, 0.011834687, 0.030601718, -0.015444895, 0.008260437, 0.014992618, 0.015457385, -0.008774464, 0.012171178, 0.0015264204, 0.025315216, -0.0047199186, 0.010815794, 0.004645248, -0.03240198, 0.033621814, 0.032036293, -0.023822723, 0.018345598, -0.0005302871, 0.0067950827, -0.015412926, -0.014910379, 0.005240889, -0.020096406, 0.0032101183, 0.013405534, -0.005293882, 0.0044164644, 0.029262709, 0.0056819436, -0.013889478, -0.0058947117, -0.039159242, 0.015333993, -0.05724625, 0.03954862, ...]",4
20,AskMen,"[-0.00092732353, 0.0030645514, 0.024757257, 0.0054917857, 0.0075081964, -0.01874744, 0.037123837, -0.013795566, 0.015251031, -0.010160403, 0.006198824, -0.0060085594, -0.0030103636, 0.015348448, 0.010909678, -0.0019617914, 0.025535518, -0.023679353, -0.021577079, 0.01950273, -0.013287892, 0.008497651, 0.014970958, -0.0028421902, -0.015570185, 0.0035307496, -0.0065879193, -0.0057931393, -0.015539492, 0.008235079, -0.02162828, 0.026916932, -0.0029244304, 0.0028267393, -0.016963553, 0.0018820948, -0.0013078743, 0.0048784516, 0.012449013, -0.0063571176, 0.0014017506, 5.8582086e-06, 0.01757279, 0.003813128, -0.010587379, 0.0003509287, 0.012031651, -0.018160192, 0.016769474, -0.017428432, -0.0061667217, -0.014912412, -0.007042869, 0.010195926, 0.014117682, 0.00586648, -0.00913574, 0.0015374006, 0.006566269, 0.006254823, -0.019042924, 0.02007505, -0.01459621, 0.013621057, 0.018432954, 0.008460761, -0.0092553245, 0.02425202, 0.0041757952, 0.032659035, -0.014282995, 0.017026361, -0.0135740535, 0.03275813, -0.015610154, 0.006618379, 0.018586727, -0.037159007, 0.025565425, 0.030684859, -0.016784167, -0.001025576, 0.0064343805, 0.0093185315, -0.0157774, -0.026558947, 0.011401212, -0.025011433, 0.0005015349, 0.0036333464, -0.03268356, -0.009433834, 0.004585114, 0.0006160571, -0.018123312, -0.010936314, -0.03277145, 0.02018474, -0.05254829, 0.048579216, ...]",4
21,AskMenOver30,"[-0.00033440115, 7.5315074e-05, 0.027577708, 0.011378074, 0.0034606406, -0.012567688, 0.017438041, -0.013919916, 0.0004571832, -0.01065009, 0.00040058937, -0.009770518, 8.872736e-05, 0.020293813, 0.006087495, 0.0075203744, 0.010946129, -0.024279198, -0.012105199, 0.00800622, -0.019480243, 0.0051570083, 0.0086863665, -0.006083225, -0.008715783, 0.016366713, -0.014496227, 0.003371099, -0.019047488, 0.010346551, -0.005875625, 0.025884904, 0.0026535438, 0.004448075, -0.005808102, 0.0015150654, -0.008312576, 0.016085135, 0.013355713, 0.0015727425, 0.009431619, 0.008936796, 0.017463312, -0.0060688998, -0.015025388, -0.005578454, 0.007826518, -0.023369178, 0.028953718, -0.009787881, -0.009488783, -0.016229961, -0.0102968495, -0.004507904, 0.0069656023, 0.019979853, 0.0004618887, 0.009591891, 0.00475586, 0.0031820904, -0.019205453, 0.016147042, -0.016962137, 0.015317831, 0.0110892225, 0.014319615, -0.0131004695, 0.014709971, -0.00078331504, 0.022105176, -0.009690974, 0.016572885, -0.00024340085, 0.030919185, -0.012652453, 0.01001007, 0.010019614, -0.029640565, 0.027822558, 0.018612135, -0.023047684, 0.004246509, -0.012080462, 0.00825575, -0.014061381, -0.024257503, 0.009768016, -0.

In [47]:
love =  list(subreddit_embeddings[subreddit_embeddings['cluster']==4]['subreddit'])
print(love)

['Advice', 'AskMen', 'AskMenOver30', 'AskWomen', 'BPD', 'BPDlovedones', 'DeadBedrooms', 'Divorce', 'DoesAnybodyElse', 'Drugs', 'EDAnonymous', 'ForeverAlone', 'GetMotivated', 'INTP', 'IncelsWithoutHate', 'JUSTNOFAMILY', 'JUSTNOMIL', 'JustNoSO', 'LGBTeens', 'LetsNotMeet', 'Marriage', 'NarcissisticAbuse', 'NoFap', 'OkCupid', 'Parenting', 'SuicideWatch', 'TooAfraidToAsk', 'TrueOffMyChest', 'Trufemcels', 'TwoXChromosomes', 'adultery', 'askwomenadvice', 'aspergers', 'bipolar', 'breakingmom', 'childfree', 'confession', 'confessions', 'dating', 'dating_advice', 'datingoverthirty', 'depression', 'drunk', 'dxm', 'entp', 'fatlogic', 'gay', 'gaybros', 'intj', 'lgbt', 'mentalhealth', 'offmychest', 'polyamory', 'raisedbynarcissists', 'rant', 'relationship_advice', 'relationships', 'seduction', 'self', 'sex', 'socialanxiety', 'sugarlifestyleforum', 'survivinginfidelity', 'traaaaaaannnnnnnnnns', 'trees']


In [48]:
l = 0
for s in love:
    l += len(get_text_from_subreddit(dataset, s))

print(l)

9026


In [49]:
clusters.loc[clusters['cluster']=='4', 'Number of samples'] = l

In [50]:
clusters

,cluster,topic label,Number of samples
0,0,Humor,13791
1,1,Other,5839
2,2,Sport,7042
3,3,Politics,10273
4,4,Love and relationship,9026
5,5,Film and Tv series,None
6,6,Videogames,None


### Cluster 5 (Film and TV series)

In [51]:
subreddit_embeddings[subreddit_embeddings['cluster']==5] #flim and TV series

,subreddit,embedding,cluster


In [52]:
# cluster 9 -> cluster 5
subreddit_embeddings.loc[subreddit_embeddings['cluster']==9, 'cluster'] = 5

In [53]:
subreddit_embeddings[subreddit_embeddings['cluster']==5] #flim and TV series

,subreddit,embedding,cluster
3,90DayFiance,"[-0.019507552, -0.00010089574, 0.018924331, -0.0038835085, 0.0060501234, -0.01569419, 0.049998507, -0.01904653, 0.0032230413, -0.019662304, -0.008358756, -0.013460865, 0.0065116957, 0.0034610345, -0.018973177, 0.004109488, 0.024398454, -0.018270606, -0.013174686, 0.020341944, -0.012314899, 0.016231775, 0.019261865, 0.015190753, -0.0032424924, 0.012099743, -0.0048506665, -0.0030736676, -0.013995789, -0.005119908, -0.0049036867, 0.04076871, -0.006272048, 0.01964927, -0.010928114, -0.010002585, 0.0069559994, 0.029052923, 0.0034749992, -0.009018267, -0.0030517401, -0.028956687, -0.0023353826, 0.0062700445, -0.02901863, -0.008297744, 0.006232326, -0.015814122, 0.022042062, -0.016543563, -0.03301168, -0.019316193, -0.0030435629, -0.014972599, -0.010391119, 0.014357637, 0.0107155405, -0.008648491, 0.017338803, 0.01906286, -0.0055567026, 0.028424773, -0.022645047, 0.022397403, 0.014496168, -0.007771397, -0.0040291203, 0.008417062, 0.0025153407, 0.020618653, 0.01665875, 0.018763488, 0.001761911, 0.013061582, -0.014513414, 0.012087444, 0.015032094, -0.029182835, 0.0187016, 0.024862831, -0.016964097, -0.01950248, 0.008608155, 0.0012633805, -0.0019364496, -0.00072301785, -0.017652422, -0.034722503, -0.0079342555, -0.009784057, -0.03788417, -0.007694937, 0.020204091, 0.0076382705, -0.015932024, -0.010579802, -0.020447962, 0.0033945518, -0.07285643, 0.052548166, ...]",5
4,90dayfianceuncensored,"[-0.016648285, -0.0016126882, 0.009415893, -0.0016100111, 0.0019142216, -0.017117219, 0.04244234, -0.015882403, 0.0043267184, -0.018114535, -0.0011434428, -0.0148694515, 0.0066956542, 0.006140645, -0.01705578, 0.011066317, 0.017125696, -0.018565735, -0.01967116, 0.015638137, -0.015192008, 0.01662443, 0.014647336, 0.00776881, -0.0048861303, 0.009418479, -0.004536365, 0.0001366113, -0.017532269, 0.00096075976, -0.007773897, 0.03767317, -0.0058949995, 0.022984963, -0.005164784, -0.0073938337, 0.011315836, 0.03042451, 0.0017337339, -0.008336772, -0.004055907, -0.029913757, 0.0015786457, 4.0773466e-05, -0.024261171, -0.0093112765, -0.0025688244, -0.017866634, 0.013873043, -0.024502411, -0.02897177, -0.026675196, 0.0012092976, -0.01912418, -0.008045133, 0.013183872, 0.013678813, -0.008394562, 0.020386884, 0.024118226, -0.007886545, 0.029661028, -0.016760131, 0.019187475, 0.014510825, -0.012754731, -0.005630248, -0.0030878584, 0.0013569504, 0.019244732, 0.020337986, 0.016748028, -0.0024019107, 0.010417762, -0.011856027, 0.008205711, 0.02631944, -0.016911784, 0.021633776, 0.018766472, -0.0137266405, -0.012217221, 0.0104871625, 0.0026082199, -0.0041602687, 0.0005365952, -0.01724035, -0.038934346, -0.010131961, -0.005289653, -0.030049814, -0.011947373, 0.017801994, 0.0123516265, -0.015830277, -0.010853867, -0.023839882, 0.008481613, -0.0698053, 0.049212627, ...]",5
31,BeautyGuruChatter,"[-0.031506933, -0.017475396, 0.02932358, -0.007933318, 0.019502014, -0.009912258, 0.037796333, -0.009632719, -0.0037243392, -0.0151805235, -0.00055297307, -0.016167087, 0.00073085056, 0.0027748083, -0.005627279, 0.022947486, 0.017047688, -0.013421294, -0.0093181515, 0.004802724, -0.013654997, 0.001289274, 0.025785988, 0.015829025, -0.016120538, 0.0085367495, -0.010298732, -0.0012481312, -0.0065265424, -0.022858, -0.0133903, 0.04980634, -0.012284523, 0.014145563, -0.01510977, -0.006150022, 0.00053092174, 0.02521634, -0.012368418, 0.00043628292, 0.0017960422, -0.017185103, -0.004913633, -0.0048562586, -0.018417783, -0.011444453, 0.008734713, -0.020612212, 0.010279107, -0.013984489, -0.030954555, -0.03723637, -0.0023211616, -0.018536828, -0.01704366, 0.012705516, 0.000648483, -0.01305845, 0.022856418, 0.011517749, -0.011837544, 0.02129442, -0.013239893, 0.013464556, 0.023640225, -0.0027025933, -0.006117381, -0.010307735, 0.007708812, 0.011014635, 0.00088818313, 0.018464983, 0.00058482913, 0.019966284, -0.009917966, 0.019429645, 0.016383266, -0.028508224, 0.006543615, 0.035645768, 0.0059404443, -0.0037266451, 0.015718576, 0.011348711, -0.

In [54]:
films = list(subreddit_embeddings[subreddit_embeddings['cluster']==5]['subreddit'])
print(films)

['90DayFiance', '90dayfianceuncensored', 'BeautyGuruChatter', 'BigBrother', 'BravoRealHousewives', 'DanLeBatardShow', 'DevilMayCry', 'DomesticGirlfriend', 'DunderMifflin', 'FlashTV', 'Kanye', 'MortalKombat', 'Music', 'NetflixBestOf', 'PandR', 'Persona5', 'RWBY', 'RedLetterMedia', 'SpoiledDragRace', 'SquaredCircle', 'TeenMomOGandTeenMom2', 'TheWalkingDeadGame', 'TwoBestFriendsPlay', 'WWE', 'arrow', 'asoiaf', 'blogsnark', 'danganronpa', 'fireemblem', 'freefolk', 'gamegrumps', 'greysanatomy', 'loveafterlockup', 'movies', 'moviescirclejerk', 'netflix', 'popheads', 'raimimemes', 'riverdale', 'rpdrcringe', 'rupaulsdragrace', 'saltierthancrait', 'shieldbro', 'starwarsspeculation', 'television', 'vanderpumprules', 'weezer']


In [55]:
l = 0
for s in films:
    l += len(get_text_from_subreddit(dataset, s))

print(l)

4987


In [56]:
clusters.loc[clusters['cluster']=='5', 'Number of samples'] = l

In [57]:
clusters

,cluster,topic label,Number of samples
0,0,Humor,13791
1,1,Other,5839
2,2,Sport,7042
3,3,Politics,10273
4,4,Love and relationship,9026
5,5,Film and Tv series,4987
6,6,Videogames,None


### Cluster 6 (Videogames)

In [58]:
subreddit_embeddings[subreddit_embeddings['cluster']==6] #videogames

,subreddit,embedding,cluster
17,Artifact,"[-0.0198666, -0.0022379616, 0.01152883, -0.026738325, 0.005456137, -0.005533063, 0.023683058, 0.00043812994, -0.00043767173, -0.0010603125, -0.020378483, -0.0069580837, -0.009220059, 0.008687565, -0.00043128064, 0.0043468373, 0.008497436, -0.018500222, -0.012610156, 0.0071502654, -0.019249985, 0.0015196075, -0.010674201, 0.0003010751, -0.011782145, -0.007023567, -0.014429361, 0.01882879, -0.011352981, -0.023777101, -0.015643151, 0.04511947, 0.012563895, 0.0011402726, -0.023952907, -0.0052734097, 0.0003162527, 0.0063878484, -0.023627277, -0.027180495, 0.0047785584, -0.0045185187, -0.011329987, 0.00949867, -0.0061824103, -0.0069062393, -0.0062262923, -0.015641527, 0.022837948, 0.0055600507, -0.017523108, -0.017168706, -0.0074256505, -0.011856682, 0.016687088, 0.019579798, -0.004839783, 0.00402621, 0.018300753, -0.0041286643, 0.00042334522, 0.001622398, -0.021779804, 0.017641734, 0.01950334, -0.006285275, 0.00802107, 0.0073556704, -0.014402541, 0.014871675, -0.0015451121, 0.01219746, 0.004047034, 0.009251943, 0.0049465587, 0.028852612, -0.015688555, -0.037930146, 0.014948681, 0.018141171, -0.004803227, -0.011899422, -0.011642317, 0.004481455, -0.00248636, -0.004511024, 0.002469585, -0.01131168, 0.018749828, -0.009774004, 0.0060710986, 0.006806218, 0.043620955, 0.02470648, -0.008841595, 0.008400932, -0.01670872, 0.0074748537, -0.0522001, 0.06458523, ...]",6
29,Battlefield,"[-0.008459435, -0.008373664, 0.009555666, -0.027318653, 0.028762814, -0.012159255, 0.005491361, -0.008138666, -0.005928842, 0.0018658652, -0.00087916333, 0.007357918, -0.005129148, 0.024429787, 0.0017641392, -0.00063874247, 0.020211862, -0.02683749, -0.006309793, 0.005574658, -0.023989705, 0.0014539086, 0.0084316395, 0.0050032884, -0.013733774, -0.004566612, -0.017649723, 0.016956417, -0.010952816, -0.020760901, -0.0129849035, 0.040322024, 0.007989358, -0.0031029263, -0.022467729, -0.008053687, 0.014389284, -0.001960128, -0.01893303, -0.0130294645, -0.0051310565, 0.0038728197, 0.010808338, 0.014488329, 0.0017486267, 0.013083702, -0.0036897303, -0.025263274, 0.034902476, 0.0006308794, -0.0037242214, -0.03390495, -0.009565788, -0.006147524, 0.006707942, 0.010005893, -0.003848868, 0.011299738, 0.023572354, 0.010350032, -0.015689617, 0.0070620324, -0.00808776, 0.013124596, 0.035617404, 0.0027559113, 0.0039101806, -0.0074982485, -0.005544108, 0.014356727, -0.0077827484, 0.01625516, -0.0018770546, 0.01062895, -0.024942901, 0.026817972, -0.011948443, -0.03539549, 0.01000765, 0.014104233, 0.00060653343, -0.00039038027, -0.014728146, 0.017210145, -0.0014929187, 0.00054171693, -0.0048868177, -0.029010588, 0.0053327144, 0.011414711, -0.01347288, 0.0018688523, 0.04237341, 0.03372249, -0.022538722, -0.009322068, -0.0048873816, 0.0062561166, -0.045820244, 0.039958064, ...]",6
30,BattlefieldV,"[-0.0061130486, 0.010125351, 0.0058964132, -0.023217155, 0.013659836, -0.013347101, 0.017119024, 0.0011825864, -0.022232337, 0.0033588442, -0.0019847502, 0.011290497, -0.005039201, 0.01191994, 0.012397225, 0.005491904, 0.015032367, -0.021340707, -0.019091075, 0.01668959, -0.027686909, 0.00384833, 0.018080864, 0.006467695, -0.020751422, -0.0074413884, -0.0056996783, 0.016180135, -0.010784246, -0.036339965, -0.020095624, 0.039388567, -0.0032537198, -0.0046402537, -0.020054022, -0.0074571036, 0.011756467, 0.010540151, -0.022032091, -0.01644989, -0.005688778, 0.0054461607, 0.018899284, 0.0138603905, -0.0050335983, 0.0033955358, -0.010369216, -0.024839828, 0.031216033, -0.005430619, -0.0062975194, -0.03162753, -0.00868917, -0.007776179, 0.022001287, 0.0054543796, -0.0063106497, 0.006995522, 0.02306939, -0.0069311815, -0.009387305, 0.0075244308, -0.01430389, 0.008510666, 0.028579148, -0.0041149817, 0.0009843493, -0.016313182, -0.015446288, 0.019161083, -0.012199738, 0.026740428, -0.0037593502, 0.016634481, -0.012083638, 0.033606462, -0.006583714, -0.03135926, 0.0131659405, 0.01436251, -0.0008920922, -0.0048139635, -0.012051235, 0.018491508, 0.005413608

In [59]:
videogames = list(subreddit_embeddings[subreddit_embeddings['cluster']==6]['subreddit'])
print(videogames)

['Artifact', 'Battlefield', 'BattlefieldV', 'Blackops4', 'Brawlhalla', 'CODZombies', 'CoDCompetitive', 'CompetitiveForHonor', 'Competitiveoverwatch', 'FortniteCompetitive', 'Games', 'GlobalOffensive', 'MLBTheShow', 'MMORPG', 'Overwatch', 'OverwatchUniversity', 'PUBGXboxOne', 'Paladins', 'RedDeadOnline', 'SSBM', 'Seaofthieves', 'Smite', 'SoulCalibur', 'StarWarsBattlefront', 'StreetFighter', 'Tekken', 'TownofSalemgame', 'assassinscreed', 'dayz', 'deadbydaylight', 'farcry', 'forhonor', 'heroesofthestorm', 'leagueoflegends', 'masseffect', 'pcgaming', 'poker', 'reddeadredemption', 'reddeadredemption2', 'truegaming']


In [60]:
l = 0
for s in videogames:
    l += len(get_text_from_subreddit(dataset, s))

print(l)

3305


In [61]:
clusters.loc[clusters['cluster']=='6', 'Number of samples'] = l

In [62]:
clusters

,cluster,topic label,Number of samples
0,0,Humor,13791
1,1,Other,5839
2,2,Sport,7042
3,3,Politics,10273
4,4,Love and relationship,9026
5,5,Film and Tv series,4987
6,6,Videogames,3305


### Results

In [63]:
l = 0
for _, row in clusters.iterrows():
    l += row['Number of samples']

print(l)

54263


In [64]:
sum = 0
for i in range(10):
    l = get_lenght(subreddit_embeddings, i)
    print(f"number of samples cluster {i} = {l}")
    sum += l 

print(sum)

number of samples cluster 0 = 13791
number of samples cluster 1 = 5839
number of samples cluster 2 = 7042
number of samples cluster 3 = 10273
number of samples cluster 4 = 9026
number of samples cluster 5 = 4987
number of samples cluster 6 = 3305
number of samples cluster 7 = 0
number of samples cluster 8 = 0
number of samples cluster 9 = 0
54263


In [67]:
subreddit_embeddings['cluster'].value_counts().sort_index()

cluster
0    112
1     68
2     62
3     89
4     65
5     47
6     40
Name: count, dtype: int64

In [ ]:
clusters.to_csv('data/clusters.csv', index=False)

In [69]:
subreddit_embeddings.to_csv('data/subreddit_embeddings.csv', index=False)